In [1]:
import torch
from transformers import pipeline

from src.data import load_qtimelines

/home/ubuntu/projects/temporal_game/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
classifier = pipeline(
    "text-classification",
    model="hugosousa/classifier_smoll_135m_b",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

In [3]:
testset = load_qtimelines(split="test")

In [4]:
testset

Dataset({
    features: ['text', 'label'],
    num_rows: 4420
})

In [5]:
preds = classifier(testset["text"], batch_size=128)

In [8]:
preds

[{'label': '=', 'score': 0.4222927987575531},
 {'label': '<', 'score': 0.3837903141975403},
 {'label': '=', 'score': 0.39999303221702576},
 {'label': '<', 'score': 0.4569115936756134},
 {'label': '=', 'score': 0.4194202423095703},
 {'label': '<', 'score': 0.3836674392223358},
 {'label': '<', 'score': 0.45568883419036865},
 {'label': '=', 'score': 0.41751909255981445},
 {'label': '=', 'score': 0.3911389708518982},
 {'label': '<', 'score': 0.4185207486152649},
 {'label': '<', 'score': 0.45363306999206543},
 {'label': '=', 'score': 0.36611631512641907},
 {'label': '=', 'score': 0.3679313063621521},
 {'label': '=', 'score': 0.39026689529418945},
 {'label': '<', 'score': 0.38447830080986023},
 {'label': '<', 'score': 0.45562055706977844},
 {'label': '<', 'score': 0.3837519884109497},
 {'label': '=', 'score': 0.36696165800094604},
 {'label': '<', 'score': 0.4548686146736145},
 {'label': '<', 'score': 0.383940190076828},
 {'label': '<', 'score': 0.4562082588672638},
 {'label': '=', 'score': 0

In [6]:
pred_labels = [p["label"] for p in preds]
pred_scores = [p["score"] for p in preds]

In [7]:
from collections import Counter

Counter(pred_labels)

Counter({'<': 2325, '=': 1358, '>': 543, '-': 194})

In [12]:
# add pred_scores  and pred_labels to testset
testset = testset.add_column("pred_labels", pred_labels)
testset = testset.add_column("pred_scores", pred_scores)
testset

Dataset({
    features: ['text', 'label', 'pred_labels', 'pred_scores'],
    num_rows: 4420
})

In [14]:
print(testset["text"][0])

Context:
Documents Creation Time: <t0>2013-03-22</t0>
Heavy <ei1>snow</ei1> is <ei2>causing</ei2> <ei3>disruption</ei3> to transport across the UK, with heavy <ei1000>rainfall</ei1000> <ei5>bringing</ei5> <ei1001>flooding</ei1001> to the south-west of England.

Question:
What is the temporal relation between the end ei3 and the end t0?

Options:
<, in case the end ei3 happens before the end t0
>, in case the end ei3 happens after the end t0
=, in case the end ei3 happens the same time as the end t0
-, in case the end ei3 happens not related to the end t0

Answer:

